# LSTM model 1 on KaggleCompetition data set 

## Importing packages 

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os.path as op
import time
import matplotlib.pyplot as plt
#Loading tensorflow and keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import plot_model
#Loading nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout,concatenate
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Loading the model 
from sklearn.model_selection import train_test_split
import sys  
sys.path.insert(0, '../../../models')
from PY.model1 import get_model1

Using TensorFlow backend.


## Paths 

In [2]:
DATA_DIR = '../../../../../Data'
CODE_DIR_H5='../../models/H5'
# Create a path for the saving location of the model
MODEL_DIR_H5 = CODE_DIR_H5 + '/model1_Kaggle.h5'

## Loading Data 

In [3]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'Snopes/Claims_to_work_with/claims.csv'))
train

label                       title  \
0          0  politics_christmas_bestbuy   
1          0  politics_christmas_bestbuy   
2          0  politics_christmas_bestbuy   
3          0  politics_christmas_bestbuy   
4          0  politics_christmas_bestbuy   
...      ...                         ...   
29237      0    medical_emergent_stapled   
29238      0    medical_emergent_stapled   
29239      0    medical_emergent_stapled   
29240      0    medical_emergent_stapled   
29241      0    medical_emergent_stapled   

                                                    text  \
0      best buy chain eschewing use word christmas 20...   
1      best buy chain eschewing use word christmas 20...   
2      best buy chain eschewing use word christmas 20...   
3      best buy chain eschewing use word christmas 20...   
4      best buy chain eschewing use word christmas 20...   
...                                                  ...   
29237  worker tore scrotum piece machinery stapled ba...   
29238  worker tore scrotum piece machinery stapled ba...   
29239  worker tore scrotum piece machinery stapled ba...   
29240  worker tore scrotum piece machinery stapled ba...   
29241  worker tore scrotum piece machinery stapled ba...   

                           author  
0      www.godlikeproductions.com  
1                   www.sjpba.net  
2               www.englisher.net  
3                        rss2.com  
4               www.englisher.net  
...                           ...  
29237        hoax.trendolizer.com  
29238                    wfmu.org  
29239                  bloooo.com  
29240               www.sjpba.net  
29241               lucim.noip.us  

[29242 rows x 4 columns]

In [4]:
train = train[train['author'].notna()]

In [5]:
train=train.reset_index()

In [6]:
#Get the Depndent feature
X_train=train.drop('label',axis=1)
y_train=train['label']

In [7]:
size_comb=[0.05,0.1, 0.15, 0.2, 0.25, 0.3]
result_list=[]

In [ ]:
#Main program 
for comb in (size_comb):
    test_size_value=comb
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=comb)
    
    # set vocabulary size
    vo_size=500
    messages=X_train.copy()
    messages.reset_index(inplace=True)
    
    # Dataset Preprocessing
    ps =PorterStemmer()
    corpus = []
    for i in range(0, len(messages)):
        print("Status: %s / %s" %(i, len(messages)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
         
    # one hot representation
    onehot_rep = [one_hot(words, vo_size) for words in corpus]
    
    
    # pad_sequences
    sent_length = 22
    embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
    print(embedded_doc)
    embedding_vector_feature = 10
    
    
    messages1=X_test.copy()
    messages1.reset_index(inplace=True)

    
    
    # Dataset Preprocessing
    
    ps1 =PorterStemmer()
    corpus1 = []
    for i in range(0, len(messages1)):
        print("Status: %s / %s" %(i, len(messages1)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',messages1['title'][i])
        review = review.lower()
        review = review.split()
    
        review = [ps1.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus1.append(review)
        
        
    
    # one hot representation
    onehot_rep1 = [one_hot(words, vo_size) for words in corpus1]
    
    
    # pad_sequences
    sent_length = 22
    embedded_doc1=pad_sequences(onehot_rep1, padding='pre', maxlen=sent_length)
    print(embedded_doc1)
    
    
    # final data for NN
    X_final_train=np.array(embedded_doc)
    y_final_train=np.array(y_train)
    X_final_train.shape,y_final_train.shape
    
    X_final_test=np.array(embedded_doc1)
    y_final_test=np.array(y_test)
    X_final_test.shape,y_final_test.shape


    
    model = Sequential()
    model.add(Embedding(vo_size,embedding_vector_feature,input_length=sent_length))
    model.add(LSTM(100))
    model.add(Dense(1,activation='sigmoid')) 
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model 
    history=model.fit(X_final_train,y_final_train,validation_split=0.1,epochs=10,batch_size=64)
    # Evaluate the model 
    results = model.evaluate(X_final_test, y_final_test, batch_size=64)
    # Stock the results 
    item=(comb, y_final_train.shape, y_final_test.shape,results)
    result_list.append(item)

[[  0   0   0 ... 280 445  55]
 [  0   0   0 ... 425 149 104]
 [  0   0   0 ... 473 363 477]
 ...
 [  0   0   0 ... 232 211 438]
 [  0   0   0 ... 216 133 311]
 [  0   0   0 ... 470 417  28]]
[[  0   0   0 ... 338 312 171]
 [  0   0   0 ... 230  41 334]
 [  0   0   0 ... 425 332 160]
 ...
 [  0   0   0 ...   0  33 471]
 [  0   0   0 ... 453 470 170]
 [  0   0   0 ... 363 157 188]]
Train on 25001 samples, validate on 2778 samples
Epoch 1/10
25001/25001 [==============================] - 16s 658us/sample - loss: 0.4908 - accuracy: 0.7621 - val_loss: 0.4305 - val_accuracy: 0.7725
Epoch 2/10
25001/25001 [==============================] - 13s 539us/sample - loss: 0.4081 - accuracy: 0.7959 - val_loss: 0.4180 - val_accuracy: 0.7876
Epoch 3/10
25001/25001 [==============================] - 13s 524us/sample - loss: 0.4040 - accuracy: 0.7993 - val_loss: 0.4174 - val_accuracy: 0.7858
Epoch 4/10
25001/25001 [==============================] - 12s 493us/sample - loss: 0.3912 - accuracy: 0.8060 - val

In [ ]:
result_list